In [1]:
import polars as pl
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
import torch.optim as optim

import math
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

: 

In [ ]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [ ]:
df_polars

In [7]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [8]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [9]:
df_polars = df_polars.drop_nulls()

In [10]:
X = df_polars.drop('label')
y = df_polars['label']       

In [11]:
X = X.to_numpy()
X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

In [12]:
# calcular número desejado de características
def calculate_desired_num_features(current_num_features):
    root = math.sqrt(current_num_features)
    desired_num_features = math.ceil(root) ** 2
    
    return desired_num_features

In [ ]:
print(X.shape)

# Treinamento

In [14]:
results = []
SAMPLE_2D_SIZE = 3

In [15]:
class CNNModel(nn.Module):
    def __init__(self, dropout_rate=0.2):
        super(CNNModel, self).__init__()
        
        # Camadas Convolucionais
        self.conv1 = nn.Conv2d(1, 64, kernel_size=2, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(64, 64, kernel_size=2, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Camada de Pooling
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Calcula o tamanho da saída após as camadas convolucionais e de pooling
        def conv2d_out_size(size, kernel_size=2, stride=1, padding=0):
            return (size - kernel_size + 2 * padding) // stride + 1
        
        size_after_conv1 = conv2d_out_size(SAMPLE_2D_SIZE, kernel_size=2)
        size_after_conv2 = conv2d_out_size(size_after_conv1, kernel_size=2)
        size_after_pool = conv2d_out_size(size_after_conv2, kernel_size=2, stride=2)
        
        # Camada Fully Connected
        self.fc1 = nn.Linear(64 * size_after_pool * size_after_pool, 64)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 1)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool1(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x.squeeze()

In [21]:
def startTrain():
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []
    num_epochs = 10  # Número de épocas definido
    
    for train_idx, test_idx in kfold.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)  # Fit apenas no treino

        # Fazer a transformação dos dados de teste com o mesmo scaler (sem "fitar" novamente)
        X_test_scaled = scaler.transform(X_test)

        # Calcular o número desejado de características e realizar o preenchimento com zeros se necessário
        desired_num_features = 9  # 4x4 = 16 características
        current_num_features = X_train_scaled.shape[1]

        if current_num_features < desired_num_features:
            padding = desired_num_features - current_num_features
            X_train_scaled = np.pad(X_train_scaled, ((0, 0), (0, padding)), 'constant')  # Preencher com zeros
            X_test_scaled = np.pad(X_test_scaled, ((0, 0), (0, padding)), 'constant')  # Preencher com zeros
        elif current_num_features > desired_num_features:
            raise ValueError("Número de características é maior que o desejado.")

        # Realizar o reshape para 4x4
        X_train_scaled = X_train_scaled.reshape(-1, 1, SAMPLE_2D_SIZE, SAMPLE_2D_SIZE)  # 1 canal, 4x4
        X_test_scaled = X_test_scaled.reshape(-1, 1, SAMPLE_2D_SIZE, SAMPLE_2D_SIZE)    # 1 canal, 4x4

        # Converter para tensores
        X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

        # Inicializar o modelo e configurá-lo para o dispositivo (GPU/CPU)
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = CNNModel(dropout_rate=0.2)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)

        # DataLoader
        batch_size = 512
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size)

        start_training = time.time()
        
        for epoch in range(num_epochs):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
        
        end_training = time.time()

        # Avaliação
        model.eval()
        all_preds = []
        all_labels = []
        
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                outputs = model(X_batch)
                preds = torch.sigmoid(outputs)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(y_batch.cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        y_pred = (all_preds >= 0.5).astype(int)
        y_true = all_labels

        # Cálculo das métricas
        confusion = confusion_matrix(y_true, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = f1_score(y_true, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0

        # Cálculo de tempos
        training_duration = end_training - start_training
        evaluation_duration = time.time() - end_training

        # Registro de resultados
        avaliacao = [
            accuracy, balanced_accuracy, precision, recall, specificity, f1, 
            false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration
        ]
        
        #print(avaliacao)
        results_fold.append(avaliacao)
        results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["2DCNN"] + mean_results.tolist())

In [ ]:
for i in range(1,2):
    startTrain()
    print(i)

In [ ]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy' , 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

In [23]:
metrics_df.write_csv(f"metrics_results/unbalanced_2DCNN_metrics_output.csv", separator=';')